In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # CUDA가 활성화 여부
print(torch.version.cuda)  # 설치된 CUDA 버전 출력
print(torch.cuda.get_device_name(0))  # 사용 중인 GPU
print("cuda" if torch.cuda.is_available() else "cpu")

2.6.0+cu118
True
11.8
NVIDIA GeForce RTX 3050
cuda


In [42]:
import pandas as pd
train_df = pd.read_csv('data/train_dataset.csv')
test_df = pd.read_csv('data/test_dataset.csv')

In [44]:
print(train_df.head())
print(test_df.head())

                                                text emotion situation
0  일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...     E18       S06
1  이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....     E18       S06
2  회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...     E18       S06
3  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...     E18       S06
4  얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...     E18       S06
                                                text emotion situation
0  이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...     E31       S06
1  회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워. 나에게 너무 크...     E31       S06
2  상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워. 무섭게 생겼는데도 업무를 보려...     E31       S06
3  이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다. 첫 직장...     E31       S06
4  직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼. 내가 낯가림이 심해서 친해질 수...     E31       S06


In [46]:
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51628 entries, 0 to 51627
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       51628 non-null  object
 1   emotion    51628 non-null  object
 2   situation  51628 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6640 non-null   object
 1   emotion    6640 non-null   object
 2   situation  6640 non-null   object
dtypes: object(3)
memory usage: 155.8+ KB
None


In [48]:
train_df['text'][0]

'일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.'

In [50]:
import requests

# 한국어 불용어 리스트 가져오기
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.txt"
response = requests.get(url)
korean_stopwords = set(response.text.split("\n"))

In [58]:
def preprocess_text_with_adverbs_and_verbs(text):
    # 텍스트가 비어있는 경우
    if not isinstance(text, str):  
        return ""
    
    parsed_text = mecab.parse(text).split("\n")[:-2]  # 마지막 줄(EOS, 공백) 제거
    tokens = [line.split("\t") for line in parsed_text if "\t" in line]
    
    # 품사 필터링 (명사, 형용사, 부사만 남기기)
    keywords = []
    for token in tokens:
        if len(token) < 2:
            continue
        word, features = token[0], token[1].split(",")
        tag = features[0]  
        lemma = features[-3]  # 원형을 추출

        # 명사(NNG, NNP) 그대로 추가
        if tag in ['NNG', 'NNP']:
            keywords.append(word)

        # 형용사(VA) 원형 변환 및 추가
        elif tag == 'VA':
            if lemma != '*' and lemma != '' and lemma != word:
                keywords.append(lemma)
            else:
                keywords.append(word + "다")

        # 부사(MAG)는 감정의 강도나 상태를 강조하므로 추가
        elif tag == 'MAG':  # 부사 태그
            keywords.append(word)

    # 중복 단어 제거 & 불용어 제거
    keywords = list(dict.fromkeys(keywords))  # 중복 제거
    keywords = [word for word in keywords if word not in stopwords and len(word) > 1]  # 불용어 제거
    
    return " ".join(keywords)

# MeCab 적용하여 텍스트 전처리
train_df['text'] = train_df['text'].apply(preprocess_text_with_adverbs_and_verbs)


In [78]:
print(train_df.head())

                               text emotion situation
0                       없다 그냥 해결 부담     E18       S06
1    급여 물가 월급 자꾸 너무 최대한 지출 억제 고정 없다     E18       S06
2  회사 신입 말투 매일 생각 스트레스 사람 억지로 거리 예의     E18       S06
3  직장 막내 이유 심부름 많다 정말 분하다 사람 이야기 방해     E18       S06
4        입사 신입 사원 무시 너무 상사 먼저 인사 매일     E18       S06


In [22]:
# import numpy as np
# print(len(train_df['emotion'].unique()), "개", end='\n\n')
# print(train_df['emotion'].value_counts().max())
# print(train_df['emotion'].value_counts().min())
# print((train_df['emotion'].value_counts().max()-train_df['emotion'].value_counts().min())/60)
# print(train_df['emotion'].value_counts())

In [24]:
# print(len(train_df['situation'].unique()), "개", end='\n\n')
# print(train_df['situation'].value_counts())

In [26]:
# import matplotlib.pyplot as plt

# # Create a figure and axes for two vertical subplots
# fig, axes = plt.subplots(2, 1, figsize=(14, 14))

# # Emotion distribution bar chart
# axes[0].bar(train_df['emotion'].value_counts().index, train_df['emotion'].value_counts().values, color='skyblue', edgecolor='black')
# axes[0].set_title('Emotion Distribution')
# axes[0].set_xlabel('Emotion')
# axes[0].set_ylabel('Count')
# axes[0].tick_params(axis='x', rotation=45)

# # Situation distribution bar chart
# axes[1].bar(train_df['situation'].value_counts().index, train_df['situation'].value_counts().values, color='salmon', edgecolor='black')
# axes[1].set_title('Situation Distribution')
# axes[1].set_xlabel('Situation')
# axes[1].set_ylabel('Count')
# axes[1].tick_params(axis='x', rotation=45)

# plt.tight_layout(pad=4.0)  # Adjust layout and padding for better label placement
# plt.show()


In [80]:
import re

def clean_text(text):
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)  # 한글, 영어, 숫자, 공백만 유지
    text = re.sub(r"\s+", " ", text).strip()  # 중복 공백 제거
    return text

train_df["text"] = train_df["text"].apply(clean_text)
test_df["text"] = test_df["text"].apply(clean_text)


In [82]:
train_df.head()

,text,emotion,situation
0,없다 그냥 해결 부담,E18,S06
1,급여 물가 월급 자꾸 너무 최대한 지출 억제 고정 없다,E18,S06
2,회사 신입 말투 매일 생각 스트레스 사람 억지로 거리 예의,E18,S06
3,직장 막내 이유 심부름 많다 정말 분하다 사람 이야기 방해,E18,S06
4,입사 신입 사원 무시 너무 상사 먼저 인사 매일,E18,S06


In [86]:
train_df['text'][0]

'없다 그냥 해결 부담'

In [88]:
from sklearn.preprocessing import LabelEncoder

# 감정 라벨 인코딩
emotion_encoder = LabelEncoder()
train_df["emotion"] = emotion_encoder.fit_transform(train_df["emotion"])
test_df["emotion"] = emotion_encoder.transform(test_df["emotion"])

# 상황 라벨 인코딩
situation_encoder = LabelEncoder()
train_df["situation"] = situation_encoder.fit_transform(train_df["situation"])
test_df["situation"] = situation_encoder.transform(test_df["situation"])


In [90]:
train_df.head()

,text,emotion,situation
0,없다 그냥 해결 부담,8,5
1,급여 물가 월급 자꾸 너무 최대한 지출 억제 고정 없다,8,5
2,회사 신입 말투 매일 생각 스트레스 사람 억지로 거리 예의,8,5
3,직장 막내 이유 심부름 많다 정말 분하다 사람 이야기 방해,8,5
4,입사 신입 사원 무시 너무 상사 먼저 인사 매일,8,5


In [92]:
import torch
import torch.nn as nn
from transformers import BertModel, AutoTokenizer

# ✅ 1. KoBERT 커스텀 모델 정의 (감정 + 상황 다중 분류)
class KoBERTMultiOutput(nn.Module):
    def __init__(self, model_name, num_emotions, num_situations):
        super(KoBERTMultiOutput, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)  # KoBERT 기본 모델
        self.dropout = nn.Dropout(0.1)  # 드롭아웃 레이어 추가
        self.emotion_classifier = nn.Linear(self.bert.config.hidden_size, num_emotions)  # 감정 분류기
        self.situation_classifier = nn.Linear(self.bert.config.hidden_size, num_situations)  # 상황 분류기

    def forward(self, input_ids, attention_mask, token_type_ids):
        # Bert 모델을 통해 출력값을 얻음
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = self.dropout(outputs.pooler_output)  # BERT 출력에서 pooling된 벡터에 드롭아웃 적용

        # 감정과 상황을 위한 로짓 분리
        emotion_logits = self.emotion_classifier(pooled_output)
        situation_logits = self.situation_classifier(pooled_output)

        # 두 가지 로짓 반환
        return emotion_logits, situation_logits


In [93]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
import torch
from sklearn.preprocessing import LabelEncoder

# ✅ 1. Tokenizer 초기화
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert",  trust_remote_code=True)  # KoBERT 모델에 맞는 토크나이저 사용

# ✅ 2. train_df에서 텍스트를 토큰화 (훈련 데이터)
train_texts = train_df["text"].tolist()  # 'text' 컬럼에 훈련 데이터가 있다고 가정
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")  # 토큰화

# ✅ 3. 라벨 데이터 (감정, 상황)
# 감정과 상황을 숫자 레이블로 변환
emotion_encoder = LabelEncoder()
situation_encoder = LabelEncoder()

emotion_labels = emotion_encoder.fit_transform(train_df['emotion'].values)  # 감정 라벨
situation_labels = situation_encoder.fit_transform(train_df['situation'].values)  # 상황 라벨

# 텐서로 변환
emotion_labels = torch.tensor(emotion_labels)
situation_labels = torch.tensor(situation_labels)

# ✅ 4. TensorDataset 생성 (입력 데이터와 레이블을 묶음)
input_ids = train_encodings['input_ids']
attention_mask = train_encodings['attention_mask']
token_type_ids = train_encodings['token_type_ids'] if 'token_type_ids' in train_encodings else torch.zeros_like(input_ids)

# TensorDataset을 이용해 훈련 데이터셋을 생성
train_dataset = TensorDataset(input_ids, attention_mask, token_type_ids, emotion_labels, situation_labels)

# ✅ 5. DataLoader 생성 (배치 크기 설정)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # 배치 크기 16으로 설정


# ✅ 6. 테스트 데이터셋 생성
test_texts = test_df["text"].tolist()  # 'text' 컬럼에 테스트 데이터가 있다고 가정
test_encodings = tokenizer(test_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")  # 토큰화

# 테스트 데이터 라벨 (감정, 상황)
emotion_labels_test = emotion_encoder.transform(test_df['emotion'].values)  # 감정 라벨
situation_labels_test = situation_encoder.transform(test_df['situation'].values)  # 상황 라벨

# 텐서로 변환
emotion_labels_test = torch.tensor(emotion_labels_test)
situation_labels_test = torch.tensor(situation_labels_test)

# ✅ 7. TensorDataset 생성 (입력 데이터와 테스트 레이블을 묶음)
input_ids_test = test_encodings['input_ids']
attention_mask_test = test_encodings['attention_mask']
token_type_ids_test = test_encodings['token_type_ids'] if 'token_type_ids' in test_encodings else torch.zeros_like(input_ids_test)

# TensorDataset을 이용해 테스트 데이터셋을 생성
test_dataset = TensorDataset(input_ids_test, attention_mask_test, token_type_ids_test, emotion_labels_test, situation_labels_test)

# ✅ 8. DataLoader 생성 (배치 크기 설정)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # 배치 크기 16으로 설정


# ✅ 9. 배치 데이터 확인 (선택 사항)
for batch in train_dataloader:
    print(batch)  # 첫 번째 배치 확인
    break  # 첫 번째 배치만 확인

for batch in test_dataloader:
    print(batch)  # 첫 번째 테스트 배치 확인
    break  # 첫 번째 배치만 확인


[tensor([[   2,  806, 5156, 1458, 1632, 1786, 6273, 6003, 7318, 3742,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 4627, 4949, 4872,  517, 6189, 7431, 7096, 3273, 2267, 5468, 2734,
         6542,  517, 7095, 5731,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1],
        [   2, 1423, 4627, 4396, 7316, 1437,  950, 4206, 3273, 3756, 3343,    3,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1

In [ ]:
import time
import torch
import torch.nn as nn
from tqdm import tqdm
from transformers import AdamW, get_scheduler
import matplotlib.pyplot as plt
import os
from transformers import AutoTokenizer
from datetime import datetime
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


# ✅ 2. 모델 생성 (train_df의 감정, 상황 레이블 수에 맞춰 설정)
num_emotions = len(train_df["emotion"].unique())  # 감정 클래스 개수
num_situations = len(train_df["situation"].unique())  # 상황 클래스 개수
model_name = "monologg/kobert"

model = KoBERTMultiOutput(model_name, num_emotions, num_situations)  # 모델 초기화

# ✅ 3. 디바이스 설정 (GPU 또는 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model.to(device)  # 모델을 설정한 디바이스로 이동

# ✅ 4. 클래스 가중치 계산
emotion_labels = train_df['emotion'].values
situation_labels = train_df['situation'].values

# 감정과 상황에 대한 클래스 가중치 계산
emotion_class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(emotion_labels), y=emotion_labels)
situation_class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(situation_labels), y=situation_labels)

# 가중치를 텐서로 변환
emotion_weights = torch.tensor(emotion_class_weights, dtype=torch.float).to(device)
situation_weights = torch.tensor(situation_class_weights, dtype=torch.float).to(device)

# ✅ 5. 손실 함수 설정 (가중치 적용)
loss_fn = nn.CrossEntropyLoss(weight=emotion_weights)  # 감정 클래스 가중치 적용
situation_loss_fn = nn.CrossEntropyLoss(weight=situation_weights)  # 상황 클래스 가중치 적용

# ✅ 6. 옵티마이저 및 학습률 스케줄러 설정
optimizer = AdamW(model.parameters(), lr=2e-5)  # AdamW 옵티마이저 사용
num_training_steps = len(train_dataloader) * 20  # 학습할 총 스텝 수 
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)  # 학습률 스케줄러 설정

# ✅ 7. 모델 학습
num_epochs = 20
train_accuracy_history = []  # 정확도 기록
train_loss_history = []  # 손실값 기록

# 얼리스탑을 위한 변수 초기화
best_accuracy = 0.0  # 가장 좋은 정확도
patience = 5  # 개선되지 않아도 허용할 에포크 수
early_stopping_counter = 0  # 얼리스탑 카운터

# 모델 체크포인트 저장을 위한 경로 설정
model_path = "kobert_emotion_situation"
os.makedirs(model_path, exist_ok=True)  # 디렉토리가 없으면 생성




# ✅ 8. 학습 시작
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0
    correct = 0
    total = 0
    epoch_accuracy = 0
    start_time = time.time()

    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for batch in progress_bar:
        # 배치 데이터를 디바이스로 이동
        input_ids, attention_mask, token_type_ids, emotion_labels, situation_labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        emotion_labels = emotion_labels.to(device)
        situation_labels = situation_labels.to(device)

        # Forward pass
        emotion_logits, situation_logits = model(input_ids, attention_mask, token_type_ids)

        # 손실 계산 (가중치 적용)
        emotion_loss = loss_fn(emotion_logits, emotion_labels)
        situation_loss = situation_loss_fn(situation_logits, situation_labels)

        loss = emotion_loss + situation_loss

        # Backward pass
        optimizer.zero_grad()  # 기울기 초기화
        loss.backward()  # 기울기 계산
        optimizer.step()  # 최적화
        lr_scheduler.step()  # 학습률 스케줄러 업데이트

        total_loss += loss.item()  # 손실값 누적

        # 정확도 계산
        emotion_pred = torch.argmax(emotion_logits, dim=1)
        situation_pred = torch.argmax(situation_logits, dim=1)

        correct += (emotion_pred == emotion_labels).sum().item()
        correct += (situation_pred == situation_labels).sum().item()
        total += emotion_labels.size(0) + situation_labels.size(0)

        # 정확도 업데이트
        epoch_accuracy = correct / total

        # 진행상황을 바에 손실값과 정확도 표시
        progress_bar.set_postfix({"Loss": loss.item(), "Accuracy": epoch_accuracy})

    avg_loss = total_loss / len(train_dataloader)  # 평균 손실 계산
    epoch_accuracy = correct / total  # 에포크별 정확도 계산

    # 에포크별 손실과 정확도 기록
    train_loss_history.append(avg_loss)
    train_accuracy_history.append(epoch_accuracy)

    epoch_time = time.time() - start_time
    print(f"✅ Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} - Accuracy: {epoch_accuracy:.4f} - Time: {epoch_time:.2f} sec")

    # 얼리스탑 조건: 성능이 개선되지 않으면 학습을 중지
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        early_stopping_counter = 0
        # 현재 모델을 체크포인트로 저장
        torch.save(model.state_dict(), f"{model_path}/best_model.pth")
        print("✅ 모델이 개선되어 체크포인트 저장!")
    else:
        early_stopping_counter += 1
        print(f"❌ 모델 성능이 개선되지 않았습니다. (이미 개선된 횟수: {early_stopping_counter})")

    # 얼리스탑 발동 여부 확인
    if early_stopping_counter >= patience:
        print(f"⚠️ 얼리스탑 발동! 성능이 개선되지 않았습니다. (최대 {patience} 에포크)")
        break  # 학습 중지

print("🎉 KoBERT 학습 완료!")

# ✅ 모델 저장 (최종 모델 가중치 및 학습 기록 포함)
torch.save(model.state_dict(), f"{model_path}/final_model.pth")  # 최종 모델 저장
torch.save({'train_loss': train_loss_history, 'train_accuracy': train_accuracy_history}, f"{model_path}/history.pth")  # 학습 기록


True


C:\Users\kj\anaconda3\Lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/20: 100%|████████████████████████████████████████| 3227/3227 [08:09<00:00,  6.59it/s, Loss=4.6, Accuracy=0.252]


✅ Epoch 1/20 - Loss: 5.4871 - Accuracy: 0.2516 - Time: 489.42 sec
✅ 모델이 개선되어 체크포인트 저장!


Epoch 2/20: 100%|███████████████████████████████████████| 3227/3227 [08:08<00:00,  6.61it/s, Loss=4.85, Accuracy=0.324]


✅ Epoch 2/20 - Loss: 4.9178 - Accuracy: 0.3239 - Time: 488.12 sec
✅ 모델이 개선되어 체크포인트 저장!


Epoch 3/20:  69%|██████████████████████████▉            | 2226/3227 [05:37<02:30,  6.64it/s, Loss=5.06, Accuracy=0.372]